In [12]:
import torch 
import torch.nn as nn

## 1. GPT Backbone

In [13]:
GPT_CONFIG_124M = {
    "vocab_size": 50257, # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768, # Embedding dimension
    "n_heads": 12, # Number of attention heads
    "n_layers": 12, # Number of layers
    "drop_rate": 0.1, # Dropout rate
    "qkv_bias": False # Query-Key-Value bias
}

- Các thành phần của _DummyGPT_ này chúng ta đều đã tìm hiểu trong các chương trước, với _DummyTransformerBlock_ là 1 khối Transformer & _DummyLayerNorm_ là 1 lớp Layer Normalization.

In [14]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = DummyLayerNorm(cfg["emb_dim"]) #B
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

class DummyTransformerBlock(nn.Module): #C
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x): #D
        return x
    
class DummyLayerNorm(nn.Module): #E
    def __init__(self, normalized_shape, eps=1e-5): #F
        super().__init__()

    def forward(self, x):
        return x


## 2. Layer Normalization

- Khởi tạo random 2 vector:

In [15]:
torch.manual_seed(123)
batch = torch.randn(2, 5)
print(batch)

tensor([[-0.1115,  0.1204, -0.3696, -0.2404, -1.1969],
        [ 0.2093, -0.9724, -0.7550,  0.3239, -0.1085]])


- Sau khi đưa qua 1 lớp _Linear_ và qua hàm kích hoạt _ReLU_:

In [16]:
layer = nn.Sequential(
    nn.Linear(5, 6),
    nn.ReLU(),
)
outputs = layer(batch)
print(outputs)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


- Lớp mạng nơ-ron vừa code ở trên bao gồm _1 lớp tuyến tính_ `Linear` đi kèm với _1 hàm kích hoạt_ `ReLU` (Rectified Linear Unit).

- Hàm `ReLU` đơn giản là sẽ chặn các giá trị đầu vào âm và chuyển chúng về 0, đảm bảo rằng đầu ra của lớp chỉ chứa các giá trị dương.

- Trước khi áp dụng `Layer Normalization` cho các outputs này, cùng tính `mean` & `variance` của các outputs này trước:

In [17]:
mean = outputs.mean(dim=-1, keepdim=True)
variance = outputs.var(dim=-1, keepdim=True)
print(f"Mean:\n {mean}")
print(f"Variance:\n {variance}")

Mean:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


- Việc sử dụng `keepdim=True` trong các phép toán như `mean` & `variance` giúp chúng ta giữ nguyên số chiều như tensor đầu vào. Ví dụ nếu không có `keepdim=True`, thì _mean_ trả về sẽ là 1 tensor 2 chiều _[0.1324, 0.2170]_ thay vì 1 ma trận 2x1 _[[0.1324], [0.2170]]_.

- Tiếp theo đây là đoạn code triển khai _layer normalization_, với công thức sử dụng của [Standardization](https://tyanfarm.github.io/tyan-blog/docs/build-llm-from-scratch/bonus-section/normalization#standardization-z-score-normalization).

In [18]:
outputs_norm = (outputs - mean) / torch.sqrt(variance)
mean = outputs_norm.mean(dim=-1, keepdim=True)
variance = outputs_norm.var(dim=-1, keepdim=True)
print(f"Normalized outputs:\n {outputs_norm}")
print(f"Mean:\n {mean}")
print(f"Variance:\n {variance}")

Normalized outputs:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean:
 tensor([[9.9341e-09],
        [0.0000e+00]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


- Giá trị `9.9341e-09` là 9.9341 * 10^-9 = 0.0000000099341. Giá trị này gần 0 nhưng không hoàn toàn bằng 0.

- Vì vậy để dễ hiển thị hơn, ta sử dụng `torch.set_printoptions(sci_mode=False)` để tắt chế độ số học.

In [19]:
torch.set_printoptions(sci_mode=False)
print(f"Normalized outputs:\n {outputs_norm}")
print(f"Mean:\n {mean}")
print(f"Variance:\n {variance}")

Normalized outputs:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean:
 tensor([[0.0000],
        [0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


- Giờ thì ta đóng gói lại thành 1 class `LayerNorm` hoàn chỉnh:

In [20]:
torch.set_printoptions(sci_mode=False)

In [21]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

- `unbiased=False`: với hàm _var()_ thì nó chia cho _n-1_ vì tuân theo _Phương sai mẫu (Sample Variance)_. Ở _layer normalization_ này ta muốn tính _Phương sai tổng thể (Population Variance)_ nên ta set __unbiased=False__ để chia cho _n_.

- `self.eps`: _epsilon_ ngăn chặn lỗi chia cho 0 khi _phương sai_ cực nhỏ.

- `self.scale` & `self.shift`: _scale_ ($\gamma$) và _shift_ ($\beta$) là hai tham số để điều chỉnh giá trị của _layer normalization_.
    + Khi bạn thực hiện chuẩn hóa (trừ mean, chia std), bạn đang ép dữ liệu về trạng thái: Trung bình = 0 và Độ lệch chuẩn = 1.

    + Tuy nhiên, trong một số trường hợp, mạng nơ-ron có thể "nhận ra" rằng việc giữ dữ liệu ở trạng thái (0, 1) này lại làm giảm khả năng học tập của nó. Vì vậy, người ta thêm vào hai tham số:

        + Scale ($\gamma$): Cho phép mô hình thay đổi độ lệch chuẩn (độ co giãn của dữ liệu).

        + Shift ($\beta$): Cho phép mô hình dịch chuyển giá trị trung bình sang một con số khác 0.

    + Vì vậy nên khởi tạo _scale_ và _shift_ là 2 tensor _1_ và _0_ với _dim_ là số chiều của _input_.

In [25]:
layer_norm = LayerNorm(emb_dim=5)
out_layer_norm = layer_norm(batch)
mean = out_layer_norm.mean(dim=-1, keepdim=True)
var = out_layer_norm.var(dim=-1, unbiased=False, keepdim=True)
print(f"Output Layer Normalization:\n {out_layer_norm}")
print(f"Mean:\n {mean}")
print(f"Variance:\n {var}")

Output Layer Normalization:
 tensor([[ 0.5528,  1.0693, -0.0223,  0.2656, -1.8654],
        [ 0.9087, -1.3767, -0.9564,  1.1304,  0.2940]], grad_fn=<AddBackward0>)
Mean:
 tensor([[-0.0000],
        [ 0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)
